# project, Text Generation using RNN  : 작사가 만들기

## 1. 데이터 읽어오기

In [18]:
import glob
import tensorflow as tf
import os, re 

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['At first I was afraid', 'I was petrified', 'I kept thinking I could never live without you']


## 2. 데이터 정제

In [19]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [20]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
    
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> at first i was afraid <end>',
 '<start> i was petrified <end>',
 '<start> i kept thinking i could never live without you <end>',
 '<start> by my side but then i spent so many nights <end>',
 '<start> just thinking how you ve done me wrong <end>',
 '<start> i grew strong <end>',
 '<start> i learned how to get along and so you re back <end>',
 '<start> from outer space <end>',
 '<start> i just walked in to find you <end>',
 '<start> here without that look upon your face i should have changed that fucking lock <end>']

In [21]:
# 토큰 개수가 15개 이상 넘어갈 경우 제외
corpus_fourteen = []
for n in range(len(corpus)):
    if corpus[n].count(" ") < 15: # 토큰 수 = 띄어쓰기 n-1
        corpus_fourteen.append(corpus[n])

print(len(corpus_fourteen)) # 156013개 출력

156013


In [22]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
# 더 잘 알기 위해 아래 문서들을 참고하면 좋습니다
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
def tokenize(corpus_fourteen):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus_fourteen)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus_fourteen)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus_fourteen)

[[   2   70  248 ...    0    0    0]
 [   2    4   53 ...    0    0    0]
 [   2    4 1077 ...    0    0    0]
 ...
 [   2    8    4 ...    0    0    0]
 [   2   44   17 ...    0    0    0]
 [   2    6  172 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fbaf21b7e50>


## 3. 평가 데이터셋 분리

In [23]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[  2  70 248   4  53 708   3   0   0   0   0   0   0   0]
[ 70 248   4  53 708   3   0   0   0   0   0   0   0   0]


In [24]:
from sklearn.model_selection import train_test_split

source = src_input
target = tgt_input
enc_train, enc_val, dec_train, dec_val = train_test_split(source, target, test_size=0.2, shuffle=True, random_state=34)

In [25]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (124810, 14)
Target Train: (124810, 14)


## 4. 딥러닝 모델 구현

In [26]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [27]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train, epochs=10)

Epoch 1/10
3901/3901 [==============================] - 312s 79ms/step - loss: 3.3709
Epoch 2/10
3901/3901 [==============================] - 311s 80ms/step - loss: 2.7271
Epoch 3/10
3901/3901 [==============================] - 311s 80ms/step - loss: 2.4579
Epoch 4/10
3901/3901 [==============================] - 310s 80ms/step - loss: 2.2076
Epoch 5/10
3901/3901 [==============================] - 309s 79ms/step - loss: 1.9977
Epoch 6/10
3901/3901 [==============================] - 308s 79ms/step - loss: 1.7978
Epoch 7/10
3901/3901 [==============================] - 308s 79ms/step - loss: 1.6351
Epoch 8/10
3901/3901 [==============================] - 309s 79ms/step - loss: 1.4918
Epoch 9/10
3901/3901 [==============================] - 309s 79ms/step - loss: 1.3659
Epoch 10/10
3901/3901 [==============================] - 309s 79ms/step - loss: 1.2686


In [31]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [46]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you too much <end> '

# project summary
- 과도한 padding 문제를 발생시키지 않기 위해 토큰 개수를 15개 미만으로 최적화하는 과정을 진행했는데 토큰화 전 띄어쓰기의 수에 맞춰 문장을 구분하는 접근법을 통해 최적화를 시켰다. 파이썬을 처음 배우는 과정에서 실제 문제에 적용하는 과정이 재밌었고, 다양한 접근법을 통해 다양하게 해결할 수 있음을 실제로 경험해보는 좋은 기회가 되었다.
- 코드를 작성하는 과정에서 다양한 문서와 블로그를 참고하게 되었는데 오류코드에 따라 해결방법, 각종 인자값을 적용해보는 과정을 통해 실질적인 작성방법을 익히게 되었다.
- RNN의 모델 구현 방식을 익히며 NLP에 흥미가 생기는 좋은 프로젝트였고, 이번 과정을 복기하며 다양한 접근방식에 대해 생각해봐야겠다.